# Boruvka's alg

In [1]:
def find_root(root, i):
    # find returns the root of a tree with the element i
    if root[i] == i:
        return i
    return find_root(root, root[i])


def merge_trees(root, rank, x, y):
    # union unite the two sets based on their rank
    xroot = find_root(root, x)
    yroot = find_root(root, y)
    if rank[xroot] < rank[yroot]:
        root[xroot] = yroot
    elif rank[xroot] > rank[yroot]:
        root[yroot] = xroot
    else:
        root[yroot] = xroot
        rank[xroot] += 1


def boruvka(nodes, edges, cols):
    # nodes - the number of nodes in the graph
    # edges - the list of edges in the graph
    # each edge is a tuple(or a list) in the following format:
    # (node1(int), node2(int), distance(float))
    # cols - the list of the names of each node
    # example: nodes = 3, cols = ['A', 'B', 'C'] meaning that
    # node "0" is 'A'
    # node "1" is 'B'
    # node "2" is 'C'
    
    # returns the tree weight and 
    # the edges in the minimum spanning tree
    
    # list of spanning tree edges
    s_edges = list()
    # list of roots of nodes
    root = list(range(nodes))
    # list of ranks of nodes
    rank = [0] * nodes

    # initializing each node as its own component
    components = nodes

    # the weight of a minimum spanning tree
    tree_weight = 0.0

    while components > 1:
        # stores cheapest edge for each component
        cheapest = [-1.0] * nodes
        # finding the cheapest edge for every component
        for i in range(len(edges)):
            u, v, w = edges[i]
            root_u = find_root(root, u)
            root_v = find_root(root, v)

            if root_u != root_v:
                if cheapest[root_u] == -1 or cheapest[root_u][2] > w:
                    cheapest[root_u] = (u, v, w)

                if cheapest[root_v] == -1 or cheapest[root_v][2] > w:
                    cheapest[root_v] = (u, v, w)

        # Consider the above picked cheapest edges and add them
        # to MST
        for node in range(nodes):

            # Check if cheapest for current set exists
            if cheapest[node] != -1:
                u, v, w = cheapest[node]
                root_u = find_root(root, u)
                root_v = find_root(root, v)

                if root_u != root_v:
                    s_edges.append((cols[u], cols[v], w))
                    tree_weight += w
                    merge_trees(root, rank, root_u, root_v)
                    print("% s-%s % f" %
                          (cols[u], cols[v], w))
                    components = components - 1

        cheapest = [-1.0] * nodes

    print("Tree weight %f" % tree_weight)
    return (tree_weight, s_edges)


# Example

In [2]:
import pandas as pd
import numpy as np

In [3]:
mat = pd.read_csv("../data/example_mat.csv", index_col="name")

In [4]:
mat

,A,B,C,D,E,F
name,,,,,,
A,1.0,0.1,-1.0,0.4,0.3,-1.0
B,0.1,1.0,-1.0,0.4,0.2,-1.0
C,-1.0,-1.0,1.0,-1.0,0.4,0.5
D,0.4,0.4,-1.0,1.0,0.4,-1.0
E,0.3,0.2,0.4,0.4,1.0,0.7
F,-1.0,-1.0,0.5,-1.0,0.7,1.0


<div><img width=200 src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c9/Multiple_minimum_spanning_trees.svg/440px-Multiple_minimum_spanning_trees.svg.png"></div>

In [5]:
def read_mat(mat):
    # reads the graph
    # and returns the list of edges
    edges = list()
    for (i, (index, row)) in enumerate(mat.iterrows()):
        for (j, vert) in enumerate(mat.columns[i+1:]):
            if (row[vert] == -1):
                continue
            edges.append((i, j + i + 1, row[vert]))
    return edges

In [6]:
edges = read_mat(mat)

In [7]:
print(edges)

[(0, 1, 0.1), (0, 3, 0.4), (0, 4, 0.3), (1, 3, 0.4), (1, 4, 0.2), (2, 4, 0.4), (2, 5, 0.5), (3, 4, 0.4), (4, 5, 0.7)]


In [8]:
nodes = len(mat.columns)
cols = mat.columns

In [9]:
weight, s_edges = boruvka(nodes, edges, cols)

A-B  0.100000
C-E  0.400000
A-D  0.400000
B-E  0.200000
C-F  0.500000
Tree weight 1.600000


In [10]:
print(weight, s_edges)

1.6 [('A', 'B', 0.1), ('C', 'E', 0.4), ('A', 'D', 0.4), ('B', 'E', 0.2), ('C', 'F', 0.5)]


### writing the result to file

In [11]:
import csv
def edges_to_csv(file_name, edges):
    with open('../data/'+file_name+'.csv', 'w') as csvfile:
        w = csv.writer(csvfile, delimiter=',')
        w.writerow(['node1', 'node2', 'dist'])
        for i in range(len(edges)):
            w.writerow(edges[i])

In [12]:
edges_to_csv('example_mst', s_edges)

# 'Real world' example

In [13]:
# mat = pd.read_csv("../data/ru_dist_mat_v1.csv")
mat = pd.read_csv("../data/ru_dist_mat_v1.csv", index_col="name")

nodes = len(mat.columns)
edges = read_mat(mat)
cols = mat.columns

In [14]:
weight, s_edges = boruvka(nodes, edges, cols)

Abakan-Chernogorsk  0.001852
Abdulino-Priyutovo  0.003880
Abinsk-Akhtyrskiy  0.001844
Achinsk-Nazarovo  0.003181
Achkhoy-Martan-Ordzhonikidzevskaya  0.003208
Adler-Sochi  0.002648
Afipskiy-Enem  0.000883
Agidel’-Neftekamsk  0.003902
Agryz-Izhevsk  0.004463
Akademgorodok-Arsen’yev  0.000000
Akhtubinsk-Znamensk  0.005754
Aksay-Sorochinsk  0.013810
Alagir-Ardon  0.001458
Alapayevsk-Rezh  0.006619
Alatyr’-Shumerlya  0.007922
Aldan-Neryungri  0.024788
Aleksandrov-Karabanovo  0.001048
Aleksandrovsk-Kizel  0.001584
Aleksandrovskoye-Zelenokumsk  0.000000
Aleksin-Protvino  0.004556
Aleysk-Barnaul  0.014216
Altuf’yevskiy-Vostochnoe Degunino  0.000190
Al’met’yevsk-Leninogorsk  0.003806
Amursk-Komsomolsk-on-Amur  0.003526
Anapa-Temryuk  0.003433
Andreyevskoye-Istra  0.000397
Angarsk-Usol’ye-Sibirskoye  0.003534
Annino-Trubchevsk  0.001236
Anzhero-Sudzhensk-Tayga  0.003778
Apatity-Kirovsk  0.001858
Aprelevka-Krasnoznamensk  0.000724
Apsheronsk-Khadyzhensk  0.002514
Aramil-Sysert’  0.002348
Armavir-

Chernogorsk-Uzhur  0.022946
Belebey-Tuymazy  0.007138
Krymsk-Slavyansk-na-Kubani  0.003351
Bogotol-Nazarovo  0.008941
Khadyzhensk-Sochi  0.007375
Krasnodar-Yasenevo  0.001327
Neftekamsk-Sarapul  0.006245
Izhevsk-Sarapul  0.007448
Akademgorodok-Krasnokamensk  0.021136
Leninsk-Znamensk  0.006291
Ardon-Beslan  0.003035
Sergach-Shumerlya  0.009258
Tynda-Zeya  0.030323
Karabanovo-Strunino  0.001375
Chusovoy-Gubakha  0.007286
Pushchino-Yasnogorsk  0.004148
Cherepanovo-Novoaltaysk  0.010873
Mar’ina Roshcha-Vostochnoe Degunino  0.000917
Al’met’yevsk-Zainsk  0.005509
Amursk-Khabarovsk  0.027027
Anapa-Novorossiysk  0.005795
Istra-Zvenigorod  0.002203
Angarsk-Shelekhov  0.004275
Pochep-Zhukovka  0.007390
Bolotnoye-Yurga  0.005418
Apatity-Monchegorsk  0.006364
Setun’-Zvenigorod  0.002092
Apsheronsk-Belorechensk  0.003031
Aramil-Yekaterinburg  0.002866
Labinsk-Novokubansk  0.005690
Novocheboksarsk-Volzhsk  0.009199
Artëm-Nakhodka  0.009969
Arzamas-Kstovo  0.009933
Aleksandrovskoye-Novopavlovsk  0.0

In [15]:
edges_to_csv('mst_boruvka', s_edges)